## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kashan,IR,33.9831,51.4364,78.49,27,3,7.14,clear sky
1,Mahébourg,MU,-20.4081,57.7000,79.09,65,40,12.66,light rain
2,Ushuaia,AR,-54.8000,-68.3000,38.86,60,20,9.22,few clouds
3,Albany,US,42.6001,-73.9662,53.80,96,0,1.63,clear sky
4,Jamestown,US,42.0970,-79.2353,63.72,100,90,5.75,mist


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 65


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pcities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]

In [5]:
# 4a. Determine if there are any empty rows.
pcities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pcities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Jamestown,US,63.72,mist,42.0970,-79.2353,
26,Castro,BR,63.52,overcast clouds,-24.7911,-50.0119,
42,Khani,GE,62.17,few clouds,41.9563,42.9566,
74,Kargil,IN,62.89,clear sky,34.5667,76.1000,
119,Plainview,US,64.45,broken clouds,40.7765,-73.4673,
144,Airai,TL,64.04,broken clouds,-8.9266,125.4092,
145,Selkirk,CA,62.08,scattered clouds,50.1436,-96.8845,
155,Esperance,AU,62.44,overcast clouds,-33.8667,121.9000,
202,Lompoc,US,61.57,clear sky,34.6391,-120.4579,
210,Altona,DE,61.14,clear sky,53.5500,9.9333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(5)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Jamestown,US,63.72,mist,42.0970,-79.2353,Hampton Inn & Suites Jamestown
26,Castro,BR,63.52,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
42,Khani,GE,62.17,few clouds,41.9563,42.9566,კურორტი ზეკარი/Resort Zekari
74,Kargil,IN,62.89,clear sky,34.5667,76.1000,Caravan Sarai
119,Plainview,US,64.45,broken clouds,40.7765,-73.4673,Four Points by Sheraton Melville Long Island


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index=False)

In [10]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(28.0, 29.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=290, point_radius=3.5)

In [11]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))